In [21]:
# Step 1: Install required libraries
!pip install opencv-python

# Step 2: Import necessary libraries
import cv2
from cv2 import dnn_superres
import os

# Step 3: Function to validate paths
def validate_path(path, is_directory=False):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Path does not exist: {path}")
    if is_directory and not os.path.isdir(path):
        raise NotADirectoryError(f"Expected a directory, but got a file: {path}")
    return path

# Step 4: Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Step 5: Define paths for input/output
input_image = "/content/drive/MyDrive/singleImage_SR/input_image/sheep.jpg"
enhanced_folder = "/content/drive/MyDrive/singleImage_SR/enhanced_image/"
os.makedirs(enhanced_folder, exist_ok=True)  # Create folder if it doesn't exist
enhanced_output = os.path.join(enhanced_folder, "enhanced_sample.jpg")

# Step 6: Load the FSRCNN model
sr = dnn_superres.DnnSuperResImpl_create()
model_path = '/content/drive/MyDrive/singleImage_SR/FSRCNN_x4.pb'
validate_path(model_path)
sr.readModel(model_path)
sr.setModel('fsrcnn', 4)

# ✅ Step 7: Force OpenCV to use CPU instead of CUDA
sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
print("Using CPU for Super-Resolution (CUDA disabled)")

# Step 8: Function to enhance an image
def enhance_image(image_path, output_path):
    validate_path(image_path)
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Error: Could not load image.")

    upscaled_image = sr.upsample(image)
    cv2.imwrite(output_path, upscaled_image)
    print(f"✅ Enhanced image saved to: {output_path}")

# Step 9: Run the enhancement
enhance_image(input_image, enhanced_output)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using CPU for Super-Resolution (CUDA disabled)
✅ Enhanced image saved to: /content/drive/MyDrive/singleImage_SR/enhanced_image/enhanced_sample.jpg


In [22]:
# Step 1: Install necessary libraries
!pip install ultralytics opencv-python

# Step 2: Import required libraries
from ultralytics import YOLO
import cv2
import os

# Step 3: Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Define paths
enhanced_image = "/content/drive/MyDrive/singleImage_SR/enhanced_image/enhanced_sample.jpg"
detected_folder = "/content/drive/MyDrive/singleImage_SR/detected_image/"
os.makedirs(detected_folder, exist_ok=True)  # Create folder if it doesn't exist
detected_output = os.path.join(detected_folder, "detected_sample.jpg")

# Step 5: Function to detect animals using YOLO
def detect_animals(image_path, output_path, conf_threshold=0.3, device="cuda"):
    model = YOLO("yolov8n.pt")

    # ✅ Use CPU if CUDA is unavailable
    if device == "cuda":
        import torch
        if not torch.cuda.is_available():
            print("CUDA not available. Switching to CPU.")
            device = "cpu"

    results = model.predict(source=image_path, conf=conf_threshold, device=device, save=False, show=False)
    annotated_image = results[0].plot()
    cv2.imwrite(output_path, annotated_image)
    print(f"✅ Detection completed. Saved to: {output_path}")

# Step 6: Run animal detection
detect_animals(enhanced_image, detected_output)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CUDA not available. Switching to CPU.

image 1/1 /content/drive/MyDrive/singleImage_SR/enhanced_image/enhanced_sample.jpg: 448x640 1 sheep, 194.5ms
Speed: 10.2ms preprocess, 194.5ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
✅ Detection completed. Saved to: /content/drive/MyDrive/singleImage_SR/detected_image/detected_sample.jpg
